In [1]:
import cross_validation
import numpy as np
import nearest_neighbors as nn
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import importlib

## Distances

In [2]:
def euclidean_distance(X, Y):
    X_square = (X*X).sum(axis=1).reshape(-1, 1)
    Y_square = (Y*Y).sum(axis=1).reshape(1, -1)
    ans = (X_square+Y_square-2*np.dot(X, Y.T))**0.5
    return ans


def cosine_distance(X, Y):
    norm_X = np.linalg.norm(X, axis=1).reshape(-1, 1)
    norm_Y = np.linalg.norm(Y, axis=1).reshape(1, -1)
    print(norm_Y)
    ans = 1-np.dot(X, Y.T)/norm_X/norm_Y
    return ans

In [3]:
X = np.array([[0, 1, 0], [1, 0, 0], [0, 1, 0]])
Y = np.array([[0, 3, 4], [0, 0, 1], [1, 0, 0], [0, 1, 0]])

## Cross-validation

In [4]:
X, y = load_digits(return_X_y=True)

In [17]:
cross = cross_validation.knn_cross_val_score(X,y,k_list=[2,3,4,5],cv=cross_validation.kfold(X.shape[0], 3),weights=False)
cross

{2: [0.9666110183639399, 0.9632721202003339, 0.9599332220367279],
 3: [0.9699499165275459, 0.9666110183639399, 0.9649415692821369],
 4: [0.9649415692821369, 0.9666110183639399, 0.9565943238731218],
 5: [0.9649415692821369, 0.9616026711185309, 0.9582637729549248]}

## KNNClassifier

In [ ]:
X, y = load_digits(return_X_y=True)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.30)

In [ ]:
importlib.reload(nearest_neighbors)
model = nearest_neighbors.KNNClassifier(1,strategy="brute",metric="cosine",test_block_size=100)


In [ ]:
from sklearn.neighbors import NearestNeighbors
NN = NearestNeighbors(X_train,algorithm="brute",metric="cosine")
NN.fit(X_test)
NN.kneighbors()

In [ ]:
def f():
    if(True):
        a=5
    print(a)
f()